In [10]:
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math
# import torch.nn.functional as F
from net import *
from utils import *
from tracker import *
import onnx


In [151]:
289//2

144

In [11]:
backbone_m = Inception3_M()
backbone_m.update_params()
backbone_q = Inception3_Q()
backbone_q.update_params()
neck_m = AdjustLayer()
neck_m.update_params()
neck_q = AdjustLayer()
neck_q.update_params()
head = STMHead()
head.update_params()

model = STMTrack(backbone_m, backbone_q, neck_m, neck_q, head)
# model.update_params()

# Convert BatchNorm to SyncBatchNorm 
# task_model = convert_model(task_model)
model_file = "new-epoch-19.pkl"

# model_file = "epoch-19.pkl"
model_state_dict = torch.load(model_file,
                        map_location=torch.device("cpu"))

# model.load_state_dict(model_state_dict['model_state_dict'])
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [12]:
class Basemodel_Q(nn.Module):
    def __init__(self):
        super(Basemodel_Q, self).__init__()
        self.backbone_q = model.basemodel_q
        self.neck_q = model.neck_q
    def forward(self, search_img):
        fq = self.backbone_q(search_img)
        fq = self.neck_q(fq)
        return fq

In [13]:
net = Basemodel_Q()

In [14]:
ONNX_FILE_PATH = "backbone_q.onnx"

In [15]:
input = torch.randn(1,3,289,289).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, input, ONNX_FILE_PATH, input_names=["search_img"], output_names=["fq"], export_params=True)

In [16]:
y = net(input)

In [17]:
y.shape

torch.Size([1, 512, 25, 25])

In [18]:
class Memorize(nn.Module):
    def __init__(self):
        super(Memorize, self).__init__()
        self.backbone_m = model.basemodel_m
        self.neck_m = model.neck_m
    def forward(self, im_crop, fg_bg_label_map):
        fm = self.backbone_m(im_crop, fg_bg_label_map)
        fm = self.neck_m(fm)
        fm = fm.permute(1, 0, 2, 3).unsqueeze(0).contiguous()  # B, C, T, H, W
        return fm

In [19]:
net = Memorize()#(model.basemodel_m,model.neck_m)

In [20]:
input = torch.ones(1,3,289,289).cuda()
fg_bg = torch.ones(1,1,289,289).cuda()
net.eval()
net.cuda()
fm = net(input,fg_bg)

In [21]:
fm.shape

torch.Size([1, 512, 1, 25, 25])

In [22]:
with open("trt/build/fm.txt") as file:
    fm_cpp = file.readlines()

fm_cpp = [float(f[:-1]) for f in fm_cpp]
fm_cpp = np.array(fm_cpp)
fm_cpp

array([ 0.13268  ,  0.0340902,  0.0776936, ..., -0.924745 , -0.753428 ,
       -0.268034 ])

In [23]:
fm_py = fm.flatten().detach().cpu().numpy()



In [24]:
np.abs(fm_py-fm_cpp).max()

5.377369385986328

In [25]:
ONNX_FILE_PATH = "memorize.onnx"
input = torch.randn(1,3,289,289).cuda()
fg_bg = torch.randn(1,1,289,289).cuda()
net.eval()
net.cuda()
torch.onnx.export(net, (input,fg_bg), ONNX_FILE_PATH, input_names=["img","fg_bg_label_map"], output_names=["fm"], export_params=True)

In [26]:
fm = net(input,fg_bg)

In [27]:
fm.shape

torch.Size([1, 512, 1, 25, 25])

In [28]:
def get_box(xy_ctr, offsets):
    offsets = offsets.permute(0, 2, 3, 1)  # (B, H, W, C), C=4
    offsets = offsets.reshape(offsets.shape[0], -1, 4)
    xy0 = (xy_ctr[:, :, :] - offsets[:, :, :2])
    xy1 = (xy_ctr[:, :, :] + offsets[:, :, 2:])
    bboxes_pred = torch.cat([xy0, xy1], 2)

    return bboxes_pred

In [29]:
hps = dict(
        total_stride=8,
        score_size=25,
        score_offset=-1,
        test_lr=0.95,
        penalty_k=0.04,
        window_influence=0.21,
        windowing="cosine",
        m_size=289,
        q_size=289,
        min_w=10,
        min_h=10,
        phase_memorize="memorize",
        phase_track="track",
        corr_fea_output=False,
        num_segments=4,
        confidence_threshold=0.6,
        gpu_memory_threshold=-1,
        search_area_factor=4.0,
        visualization=False )

In [138]:
class Head(nn.Module):
    def __init__(self,head):
        super(Head, self).__init__()
        self.head = head
        self.total_stride = 8
        score_offset = (hps["q_size"] - 1.0 - (hps["score_size"] - 1) * hps["total_stride"]) // 2.0
        self.fm_ctr = get_xy_ctr_np(hps["score_size"], score_offset, hps["total_stride"])
    def forward(self,fm,fq):
#         q_size = 299
#         fm = fm.permute(0, 2,1, 3, 4)
        y = self.head.memory_read(fm, fq)
        cls_score, ctr_score, offsets = self.head.solve(y)
        
        cls_score = cls_score.permute(0, 2, 3, 1)
        cls_score = cls_score.reshape(cls_score.shape[0], -1, 1)

        ctr_score = ctr_score.permute(0, 2, 3, 1)
        ctr_score = ctr_score.reshape(ctr_score.shape[0], -1, 1)

        offsets = torch.exp(self.head.si * offsets + self.head.bi) * self.total_stride

        
        fm_ctr = self.fm_ctr.to(offsets.device)
        bbox = get_box(fm_ctr, offsets)
        fcos_cls_score_final = cls_score
        fcos_ctr_score_final = ctr_score
        fcos_bbox_final = bbox

        
        fcos_cls_prob_final = torch.sigmoid(fcos_cls_score_final)
        fcos_ctr_prob_final = torch.sigmoid(fcos_ctr_score_final)
        # apply centerness correction
        fcos_score_final = fcos_cls_prob_final * fcos_ctr_prob_final
        
        return fcos_score_final, fcos_bbox_final, fcos_cls_prob_final, fcos_ctr_prob_final


In [139]:
net = Head(model.head)
net.eval()
net.cuda()
ONNX_FILE_PATH = "head.onnx"
fm = torch.randn(1, 512, 6, 25, 25).cuda()
fq = torch.randn(1, 512, 25, 25).cuda()


In [140]:
torch.onnx.export(net, (fm,fq), 
                  ONNX_FILE_PATH, input_names=["fm","fq"], 
                  output_names=["fcos_score_final", "fcos_bbox_final",
                                "fcos_cls_prob_final","fcos_ctr_prob_final"]
                  , export_params=True)

In [141]:
fm = torch.ones(1, 512, 6, 25, 25).cuda()
fq = torch.ones(1, 512, 25, 25).cuda()

In [136]:
fcos_score_final, fcos_bbox_final, fcos_cls_prob_final, fcos_ctr_prob_final = net(fm,fq)

In [137]:
fcos_score_final.shape

torch.Size([1, 625, 1])

In [86]:
score, box, cls, ctr = net(fm,fq)

In [87]:
box = tensor_to_numpy(box[0]) # 1,625,4 -> 625,4
score = tensor_to_numpy(score[0])[:, 0]
cls = tensor_to_numpy(cls[0])
ctr = tensor_to_numpy(ctr[0])

In [88]:
box_wh = xyxy2cxywh(box)

In [89]:
score_size = 25
window = np.outer(np.hanning(score_size), np.hanning(score_size))
# window = window.reshape(-1)

In [90]:
window.shape

(25, 25)

In [91]:
xyxy = np.array([[1,2,3,5]])

In [92]:
xyxy2cxywh(xyxy)

array([[2. , 3.5, 3. , 4. ]], dtype=float32)

In [93]:
def _postprocess_score(score, box_wh, target_sz, scale_x, im_x_crop):
        r"""
        Perform SiameseRPN-based tracker's post-processing of score
        :param score: (HW, ), score prediction
        :param box_wh: (HW, 4), cxywh, bbox prediction (format changed)
        :param target_sz: previous state (w & h)
        :param scale_x:
        :return:
            best_pscore_id: index of chosen candidate along axis HW
            pscore: (HW, ), penalized score
            penalty: (HW, ), penalty due to scale/ratio change
        """
        def change(r):
            return np.maximum(r, 1. / r)

        def sz(w, h):
            pad = (w + h) * 0.5
            sz2 = (w + pad) * (h + pad)
            return np.sqrt(sz2)

        def sz_wh(wh):
            pad = (wh[0] + wh[1]) * 0.5
            sz2 = (wh[0] + pad) * (wh[1] + pad)
            return np.sqrt(sz2)

        # size penalty
        penalty_k = self.default_hyper_params['penalty_k']
        target_sz_in_crop = target_sz * scale_x
        s_c = change(
            sz(box_wh[:, 2], box_wh[:, 3]) /
            (sz_wh(target_sz_in_crop)))  # scale penalty
        r_c = change((target_sz_in_crop[0] / target_sz_in_crop[1]) /
                     (box_wh[:, 2] / box_wh[:, 3]))  # ratio penalty
        penalty = np.exp(-(r_c * s_c - 1) * penalty_k)
        pscore = penalty * score
        # if self._hp_visualization:
        #     vsm.visualize(pscore, self._hp_score_size, im_x_crop, self._state['cur_frame_idx'], 'pscore_0')

        # ipdb.set_trace()
        # cos window (motion model)
        window_influence = self.default_hyper_params['window_influence']
        pscore = pscore * (
            1 - window_influence) + self._state['window'] * window_influence
        best_pscore_id = np.argmax(pscore)
        # if self._hp_visualization:
        #     vsm.visualize(pscore, self._hp_score_size, im_x_crop, self._state['cur_frame_idx'], 'pscore_1')

        return best_pscore_id, pscore, penalty


In [94]:
best_pscore_id, pscore, penalty = _postprocess_score(
             score, box_wh, target_sz, scale_q, im_q_crop)

NameError: name 'target_sz' is not defined

In [95]:
# torch.onnx.export(net, (fm,fq), 
#                   ONNX_FILE_PATH, input_names=["fm","fq"], 
#                   output_names=["cls_score", "ctr_score", "offsets"], export_params=True)

In [96]:
cls_score.shape

NameError: name 'cls_score' is not defined

In [ ]:
ctr_score.shape

In [ ]:
offsets.shape

In [143]:
fcos_score_final, fcos_bbox_final, fcos_cls_prob_final, fcos_ctr_prob_final = net(fm,fq)

In [144]:
fcos_score_final_cpp = []
with open("test_trt/build/fcos_score_final.txt") as f:
    fcos_score_final_cpp = f.readlines()
    


In [145]:
fcos_score_final_cpp = [float(c[:-1]) for c in fcos_score_final_cpp]

In [146]:
fcos_score_final_cpp = np.array(fcos_score_final_cpp)

In [147]:
cls_score_py = fcos_score_final.flatten().cpu().detach().numpy()

In [148]:
np.abs(fcos_score_final_cpp-cls_score_py).max()

0.0003914305047035202

In [149]:
fcos_score_final_cpp

array([0.0166244 , 0.0145289 , 0.014197  , 0.0118082 , 0.0100687 ,
       0.00911935, 0.00892668, 0.00893894, 0.00893894, 0.00893894,
       0.00893894, 0.00893894, 0.00893894, 0.00893894, 0.00893894,
       0.00893894, 0.00893894, 0.00893894, 0.00953159, 0.0118436 ,
       0.0141428 , 0.0134514 , 0.015893  , 0.0151154 , 0.0240744 ,
       0.00693321, 0.00442553, 0.00441077, 0.00350943, 0.00311705,
       0.00271675, 0.0025655 , 0.00255595, 0.00255595, 0.00255595,
       0.00255595, 0.00255595, 0.00255595, 0.00255595, 0.00255595,
       0.00255595, 0.00255595, 0.00255595, 0.00280545, 0.00362407,
       0.00450457, 0.00427143, 0.00593123, 0.00673415, 0.012616  ,
       0.0137366 , 0.0137018 , 0.0153999 , 0.0131583 , 0.0128968 ,
       0.0125322 , 0.0122018 , 0.0121503 , 0.0121503 , 0.0121503 ,
       0.0121503 , 0.0121503 , 0.0121503 , 0.0121503 , 0.0121503 ,
       0.0121503 , 0.0121503 , 0.0121503 , 0.0134871 , 0.0176424 ,
       0.0204254 , 0.0165065 , 0.017029  , 0.0164425 , 0.01974

In [150]:
cls_score_py

array([0.01669263, 0.01453012, 0.01421366, 0.0117822 , 0.01007301,
       0.00911173, 0.00888837, 0.00887569, 0.00887569, 0.00887569,
       0.00887569, 0.00887569, 0.00887569, 0.00887569, 0.00887569,
       0.00887569, 0.00887569, 0.00887569, 0.00955623, 0.01179801,
       0.01410357, 0.01343781, 0.01595707, 0.01515085, 0.02406023,
       0.00685216, 0.00438366, 0.0043722 , 0.00350442, 0.0030963 ,
       0.00271182, 0.00256876, 0.00255117, 0.00255117, 0.00255117,
       0.00255117, 0.00255117, 0.00255117, 0.00255117, 0.00255117,
       0.00255117, 0.00255117, 0.00255117, 0.0028011 , 0.00363157,
       0.00448919, 0.00422604, 0.00592476, 0.00671855, 0.0126078 ,
       0.01374432, 0.01374402, 0.0154052 , 0.0131534 , 0.0129269 ,
       0.01255038, 0.01222945, 0.01213448, 0.01213448, 0.01213448,
       0.01213448, 0.01213448, 0.01213448, 0.01213448, 0.01213448,
       0.01213448, 0.01213448, 0.01213448, 0.01353282, 0.01760681,
       0.02037367, 0.01647942, 0.0170403 , 0.01643598, 0.01975